#Import library

In [1]:
## single frame analysis: whether there are large spatta?
import cv2
from IPython import display
from matplotlib import pyplot as plt
import time
from numpy import dtype,uint8
import scipy.ndimage as ndimage
import glob
import pandas as pd
import numpy as np
import os

#Check number of images

#20220401

In [3]:
dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401'
def checkNum(dir=dir,date_20221020=False):
    for i in range(1,13):
        for j in range(1,3):
            if date_20221020 :
                src_img_dir = os.path.join(dir,"{}\crop".format((i-1)*2+j+20))
            else:
                #get num of file in a directory
                src_img_dir = os.path.join(dir,"{}_{}\crop".format(i,j))

            num_photo = sum(os.path.isfile(os.path.join(src_img_dir,name)) for name in os.listdir(src_img_dir))

            print(f"({i},{j})::",num_photo)

checkNum()

(1,1):: 540
(1,2):: 537
(2,1):: 1080
(2,2):: 1079
(3,1):: 2162
(3,2):: 2162
(4,1):: 540
(4,2):: 540
(5,1):: 1080
(5,2):: 1081
(6,1):: 2163
(6,2):: 2162
(7,1):: 540
(7,2):: 540
(8,1):: 1082
(8,2):: 1079
(9,1):: 2161
(9,2):: 2162
(10,1):: 540
(10,2):: 540
(11,1):: 1080
(11,2):: 1080
(12,1):: 2161
(12,2):: 2092


#20220603

In [4]:
DIR_20220603 = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220603'
checkNum(dir = DIR_20220603)

(1,1):: 540
(1,2):: 533
(2,1):: 1079
(2,2):: 1080
(3,1):: 2162
(3,2):: 2163
(4,1):: 457
(4,2):: 540
(5,1):: 1079
(5,2):: 1080
(6,1):: 2162
(6,2):: 2162
(7,1):: 541
(7,2):: 540
(8,1):: 1080
(8,2):: 1080
(9,1):: 2162
(9,2):: 2163
(10,1):: 539
(10,2):: 539
(11,1):: 1080
(11,2):: 1079
(12,1):: 2162
(12,2):: 2162


#20221020

In [5]:
DIR_20221020 = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20221020'
checkNum(dir = DIR_20221020,date_20221020=True)

(1,1):: 244
(1,2):: 448
(2,1):: 895
(2,2):: 245
(3,1):: 448
(3,2):: 863
(4,1):: 235
(4,2):: 430
(5,1):: 857
(5,2):: 233
(6,1):: 312
(6,2):: 880
(7,1):: 236
(7,2):: 434
(8,1):: 866
(8,2):: 240
(9,1):: 440
(9,2):: 883
(10,1):: 240
(10,2):: 442
(11,1):: 889
(11,2):: 242
(12,1):: 447
(12,2):: 892


#concatenate imgs

In [6]:
def concatImg(x_patch = 3, y_patch = 3,INVERSE = True):
    h,w,c = 80,80,1
    H = h*3
    W = w*3
    concat = np.zeros((H,W,c))
    folder_path = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401\1_1\crop'
    for i in range(9):
        file = "{:04d}.jpg".format(i+10)
        path = os.path.join(folder_path,file)
        img = cv2.imread(path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = np.expand_dims(np.array(img).astype("int32"),2)
        #new to old
        if INVERSE:
            row =  (x_patch-1) - i//3
            column = (y_patch-1) - i%3
            concat[h*row:h*(row+1),w*column:w*(column+1),:] = img
        #old to new
        else:
            row = i//3
            column = i%3
            concat[h*row:h*(row+1),w*column:w*(column+1),:] = img
    cv2.imwrite(r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401\concat_inverse.jpg',concat)

concatImg(INVERSE=True)

In [18]:
import numpy as np
import os
import csv
import random

class Dataset():
  def __init__(self,x_patch = 3, y_patch = 3, height = 80,width = 80,
                date = "20220401",
                src_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401',
                dir_concatImg = "concat_vit"
              ):
    #num of patches
    self.x_patch = x_patch
    self.y_patch = y_patch  
    #size for 1 patch
    self.height = height
    self.width = width
    #size for concatenated img
    self.H = x_patch*height
    self.W = y_patch*width
    #date of dataset
    self.date = date
    #img src directory
    self.src_dir = src_dir
    #directory for concat img
    self.dir_concatImg = dir_concatImg
    #load spatter labels from .txt file
    if date == "20220401":
        dir=r'C:\Users\Furukawa\Documents\TechAssistant\Video\20220401\video'
        self.spatter_data_1_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(1,1)), dtype='int').tolist() #in coping with text file which divide data with space, remove delimiter params. 
        self.spatter_data_1_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(1,2)), dtype='int').tolist()
        self.spatter_data_2_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(2,1)), dtype='int').tolist()
        self.spatter_data_2_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(2,2)), dtype='int').tolist()
        self.spatter_data_3_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(3,1)), dtype='int').tolist()
        self.spatter_data_3_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(3,2)), dtype='int').tolist()
        self.spatter_data_4_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(4,1)), dtype='int').tolist()
        self.spatter_data_4_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(4,2)), dtype='int').tolist()
        self.spatter_data_5_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(5,1)), dtype='int').tolist()
        self.spatter_data_5_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(5,2)), dtype='int').tolist()
        self.spatter_data_6_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(6,1)), dtype='int').tolist()
        self.spatter_data_6_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(6,2)), dtype='int').tolist()
        self.spatter_data_7_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(7,1)), dtype='int').tolist()
        self.spatter_data_7_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(7,2)), dtype='int').tolist()
        self.spatter_data_8_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(8,1)), dtype='int').tolist()
        self.spatter_data_8_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(8,2)), dtype='int').tolist()
        self.spatter_data_9_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(9,1)), dtype='int').tolist()
        self.spatter_data_9_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(9,2)), dtype='int').tolist()
        self.spatter_data_10_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(10,1)), dtype='int').tolist()
        self.spatter_data_10_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(10,2)), dtype='int').tolist()
        self.spatter_data_11_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(11,1)),dtype='int').tolist()
        self.spatter_data_11_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(11,2)), dtype='int').tolist()
        self.spatter_data_12_1 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(12,1)), dtype='int').tolist()
        self.spatter_data_12_2 = np.loadtxt(os.path.join(dir,'220401_xiQ_{}-{}_spatter.txt'.format(12,2)), dtype='int').tolist()

    if date == "20220603":
        dir = r'C:\Users\Furukawa\Documents\TechAssistant\Video\20220603\video'
        head  = "xiQ_20220603"
        self.spatter_data_1_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,1,1)), dtype='int').tolist()
        self.spatter_data_1_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,1,2)), dtype='int').tolist()
        self.spatter_data_2_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,2,1)), dtype='int').tolist()
        self.spatter_data_2_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,2,2)), dtype='int').tolist()
        self.spatter_data_3_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,3,1)), dtype='int').tolist()
        self.spatter_data_3_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,3,2)), dtype='int').tolist()
        self.spatter_data_4_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,4,1)), dtype='int').tolist()
        self.spatter_data_4_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,4,2)), dtype='int').tolist()
        self.spatter_data_5_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,5,1)), dtype='int').tolist()
        self.spatter_data_5_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,5,2)), dtype='int').tolist()
        self.spatter_data_6_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,6,1)), dtype='int').tolist()
        self.spatter_data_6_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,6,2)), dtype='int').tolist()
        self.spatter_data_7_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,7,1)), dtype='int').tolist()
        self.spatter_data_7_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,7,2)), dtype='int').tolist()
        self.spatter_data_8_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,8,1)), dtype='int').tolist()
        self.spatter_data_8_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,8,2)), dtype='int').tolist()
        self.spatter_data_9_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,9,1)), dtype='int').tolist()
        self.spatter_data_9_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,9,2)), dtype='int').tolist()
        self.spatter_data_10_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,10,1)), dtype='int').tolist()
        self.spatter_data_10_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,10,2)), dtype='int').tolist()
        self.spatter_data_11_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,11,1)), dtype='int').tolist()
        self.spatter_data_11_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,11,2)), dtype='int').tolist()
        self.spatter_data_12_1 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,12,1)), dtype='int').tolist()
        self.spatter_data_12_2 = np.loadtxt(os.path.join(dir,'{}_{}-{}_spatter.txt'.format(head,12,2)), dtype='int').tolist()
    
    if date == "20221020":
        dir = r'C:\Users\Furukawa\Documents\TechAssistant\Video\20221020\video'
        head  = "XIMEA_221020"
        self.spatter_data_1_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,21)), dtype='int').tolist()
        self.spatter_data_1_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,22)), dtype='int').tolist()
        self.spatter_data_2_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,23)), dtype='int').tolist()
        self.spatter_data_2_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,24)), dtype='int').tolist()
        self.spatter_data_3_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,25)), dtype='int').tolist()
        self.spatter_data_3_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,26)), dtype='int').tolist()
        self.spatter_data_4_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,27)), dtype='int').tolist()
        self.spatter_data_4_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,28)), dtype='int').tolist()
        self.spatter_data_5_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,29)), dtype='int').tolist()
        self.spatter_data_5_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,30)), dtype='int').tolist()
        self.spatter_data_6_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,31)), dtype='int').tolist()
        self.spatter_data_6_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,32)), dtype='int').tolist()
        self.spatter_data_7_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,33)), dtype='int').tolist()
        self.spatter_data_7_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,34)), dtype='int').tolist()
        self.spatter_data_8_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,35)), dtype='int').tolist()
        self.spatter_data_8_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,36)), dtype='int').tolist()
        self.spatter_data_9_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,37)), dtype='int').tolist()
        self.spatter_data_9_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,38)), dtype='int').tolist()
        self.spatter_data_10_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,39)), dtype='int').tolist()
        self.spatter_data_10_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,40)), dtype='int').tolist()
        self.spatter_data_11_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,41)), dtype='int').tolist()
        self.spatter_data_11_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,42)), dtype='int').tolist()
        self.spatter_data_12_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,43)), dtype='int').tolist()
        self.spatter_data_12_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,44)), dtype='int').tolist()

  def concatImg(self,index,src_img_dir,save_dir,INVERSE = False):
      """concatenate images
      Args:
        index: index for spatter frame, so index is the latest index in 9 frames, ie. index, index-1,index-2,..., index-8
        src_img_dir : source directory of cropped imgs
        save_dir : save dir for concatenate images
        x_patch, y_patch : number of patches in x and y axis
        INVERSE : patch order. If INVERSE is True, new to old is applied
      Return:
        file_path : path for concat img
        write concatenated images into designated directory
      """
      concat = np.zeros((self.H,self.W,1))
      for i in range(9):
          file = "{:04d}.jpg".format(index-i) #from new to old
          path = os.path.join(src_img_dir,file)
          img = cv2.imread(path)
          img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #gray scale
          img = np.expand_dims(np.array(img).astype("int32"),2) #add color channel : (height,width,1)
          #new to old
          if INVERSE:
              row = i//3
              column = i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
          #old to new
          else:
              row =  (self.x_patch-1) - i//3
              column = (self.y_patch-1) - i%3
              concat[self.height*row:self.height*(row+1),self.width*column:self.width*(column+1),:] = img
      file_path = os.path.join(save_dir,f"{index:04d}.jpg")
      cv2.imwrite(file_path,concat)
      return file_path

  def checkImg(self,file_path):
    """check whether laser process has started

    Args:
        file_path (str): img file path
    """
    img = cv2.imread(file_path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = gray
    ret,thresh = cv2.threshold(gray,60,255,cv2.THRESH_BINARY)
    #find contours
    contours, hierarchy = cv2.findContours(np.array(thresh,dtype=uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    Area = []
    #detect laser area
    if (bool(contours)==True):
      #面積(px*px)
      for j in contours:
        area = cv2.contourArea(j)
        Area.append(area)

      if Area:
        area_max = np.max(Area)
        if area_max > 64:
           ret = True
           return ret
        else:
           ret = False
           return ret
      else:
         ret = False
         return ret
      
  # adjust the number of no spatter below 51
  def make(self,filename,filename_test,div,sec):
    #フォルダに存在するファイルを取得する、0001.jpg~0005.jpg→{}/*.jpg
    
    #spatter label 
    if self.date == "20220401":
      spatter = [[[],[]],
                [self.spatter_data_1_1,self.spatter_data_1_2],
                [self.spatter_data_2_1,self.spatter_data_2_2],
                [self.spatter_data_3_1,self.spatter_data_3_2],
                [self.spatter_data_4_1,self.spatter_data_4_2],
                [self.spatter_data_5_1,self.spatter_data_5_2],
                [self.spatter_data_6_1,self.spatter_data_6_2],
                [self.spatter_data_7_1,self.spatter_data_7_2],
                [self.spatter_data_8_1,self.spatter_data_8_2],
                [self.spatter_data_9_1,self.spatter_data_9_2],
                [self.spatter_data_10_1,self.spatter_data_10_2],
                [self.spatter_data_11_1,self.spatter_data_11_2],
                [self.spatter_data_12_1,self.spatter_data_12_2]]
    elif self.date == "20220603":
       spatter = [[[],[]],
                [self.spatter_data_1_1,self.spatter_data_1_2],
                [self.spatter_data_2_1,self.spatter_data_2_2],
                [self.spatter_data_3_1,self.spatter_data_3_2],
                [[self.spatter_data_4_1],self.spatter_data_4_2],
                [self.spatter_data_5_1,self.spatter_data_5_2],
                [self.spatter_data_6_1,self.spatter_data_6_2],
                [[self.spatter_data_7_1],[self.spatter_data_7_2]],
                [self.spatter_data_8_1,self.spatter_data_8_2],
                [self.spatter_data_9_1,self.spatter_data_9_2],
                [self.spatter_data_10_1,self.spatter_data_10_2],
                [self.spatter_data_11_1,self.spatter_data_11_2],
                [self.spatter_data_12_1,self.spatter_data_12_2]]
    elif self.date == "20221020":
       spatter = [[[],[]],
                [self.spatter_data_1_1,self.spatter_data_1_2],
                [[self.spatter_data_2_1],self.spatter_data_2_2],
                [self.spatter_data_3_1,[self.spatter_data_3_2]],
                [[self.spatter_data_4_1],[self.spatter_data_4_2]],
                [self.spatter_data_5_1,[self.spatter_data_5_2]],
                [self.spatter_data_6_1,self.spatter_data_6_2],
                [self.spatter_data_7_1,[self.spatter_data_7_2]],
                [[self.spatter_data_8_1],[self.spatter_data_8_2]],
                [[self.spatter_data_9_1],self.spatter_data_9_2],
                [self.spatter_data_10_1,[self.spatter_data_10_2]],
                [self.spatter_data_11_1,[self.spatter_data_11_2]],
                [self.spatter_data_12_1,self.spatter_data_12_2]]
    
    if self.date == "20220401"  or self.date == "20220603":
      #set src img directory
      src_img_dir = os.path.join(self.src_dir,"{}_{}\crop".format(div,sec))
      #make save directory
      save_dir= os.path.join(self.src_dir,"{}_{}\{}".format(div,sec,self.dir_concatImg))
      if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    elif self.date == "20221020":
      #set src img directory
      src_img_dir = os.path.join(self.src_dir,"{}\crop".format((div-1)*2+sec+20))
      #make save directory
      save_dir= os.path.join(self.src_dir,"{}\{}".format(((div-1)*2+sec+20),self.dir_concatImg))
      if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    #spatter count
    count_spatter = 0
    count_nonspatter = 0
    #get num of file in a directory
    num_photo = sum(os.path.isfile(os.path.join(src_img_dir,name)) for name in os.listdir(src_img_dir))
    print("{}-{} progressing :: {}".format(div,sec,num_photo))
    #make dataset
    for i in range(10,num_photo-10):
      file_path = os.path.join(src_img_dir,f"{i:04d}.jpg")
      ret = self.checkImg(file_path=file_path)
      #laser welding has started
      if ret :
        #spater frame
        if (((i+10) in spatter[div-1][sec-1]) or ((i+9) in spatter[div-1][sec-1]) or ((i+8) in spatter[div-1][sec-1]) or 
            ((i+7) in spatter[div-1][sec-1]) or ((i+6) in spatter[div-1][sec-1]) or ((i+5) in spatter[div-1][sec-1]) or 
            ((i+4) in spatter[div-1][sec-1]) or ((i+3) in spatter[div-1][sec-1]) or ((i+2) in spatter[div-1][sec-1])):  
          #whether spatter frame is included in input frame
          if ((i not in spatter[div-1][sec-1]) and ((i-1) not in spatter[div-1][sec-1]) and ((i-2) not in spatter[div-1][sec-1]) and 
              ((i-3) not in spatter[div-1][sec-1]) and ((i-4) not in spatter[div-1][sec-1]) and ((i-5) not in spatter[div-1][sec-1]) and 
            ((i-6) not in spatter[div-1][sec-1]) and ((i-7) not in spatter[div-1][sec-1]) and ((i-8) not in spatter[div-1][sec-1])) :
            #concatenate images
            file_path = self.concatImg(index=i,src_img_dir=src_img_dir,save_dir=save_dir,INVERSE = False)
            if random.random() >= 0.1: #85% is train data
              with open(filename,"a",newline='') as f:
                writer = csv.writer(f)
                writer.writerow([1,file_path])
            else:
              with open(filename_test,"a",newline='') as f:
                  writer = csv.writer(f)
                  writer.writerow([1,file_path])
            count_spatter += 1
        #non-spatter frame
        else:
          #whether spatter frame is included in input frame
          if ((i not in spatter[div-1][sec-1]) and ((i-1) not in spatter[div-1][sec-1]) and ((i-2) not in spatter[div-1][sec-1]) and 
              ((i-3) not in spatter[div-1][sec-1]) and ((i-4) not in spatter[div-1][sec-1]) and ((i-5) not in spatter[div-1][sec-1]) and 
            ((i-6) not in spatter[div-1][sec-1]) and ((i-7) not in spatter[div-1][sec-1]) and ((i-8) not in spatter[div-1][sec-1])) :
            #random function for variety of dataset
            if random.random() >= 0.85:
              #concatenate images
              file_path = self.concatImg(index=i,src_img_dir=src_img_dir,save_dir=save_dir,INVERSE = False)
              if random.random() >= 0.1: #85% is train data
                with open(filename,"a",newline='') as f:
                  writer = csv.writer(f)
                  writer.writerow([0,file_path])
              else:
                with open(filename_test,"a",newline='') as f:
                  writer = csv.writer(f)
                  writer.writerow([0,file_path])
              count_nonspatter += 1
    print(f"Finished! Spatter :: {count_spatter}, Non-spatter :: {count_nonspatter}")

#20220401

#Initialize dataset file

In [19]:
import csv







#Initialize csv dataset?
INITIALIZE = True



#prepare csv files
file_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\csv\ViT'
filename=os.path.join(file_dir,"train_vit.csv")
filename_test=os.path.join(file_dir,"test_vit.csv")
#///////////////////
if INITIALIZE:
  #make csv file
  with open(filename,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Spatter","Path"])

  with open(filename_test,"w") as f:
    writer = csv.writer(f)
    writer.writerow(["Spatter","Path"])

In [21]:
dataset = Dataset(x_patch = 3, y_patch = 3, height = 80,width = 80,
                date = "20220401",
                src_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220401',
                dir_concatImg = "concat_vit"
                )

#make dataset one by one
for i in range(1,13):
  for j in range(1,3):
    dataset.make(filename=filename,filename_test=filename_test,div = i,sec = j)

1-1 progressing :: 540
Finished! Spatter :: 0, Non-spatter :: 55
1-2 progressing :: 537
Finished! Spatter :: 0, Non-spatter :: 68
2-1 progressing :: 1080
Finished! Spatter :: 36, Non-spatter :: 134
2-2 progressing :: 1079
Finished! Spatter :: 28, Non-spatter :: 110
3-1 progressing :: 2162
Finished! Spatter :: 105, Non-spatter :: 243
3-2 progressing :: 2162
Finished! Spatter :: 125, Non-spatter :: 225
4-1 progressing :: 540
Finished! Spatter :: 43, Non-spatter :: 37
4-2 progressing :: 540
Finished! Spatter :: 31, Non-spatter :: 56
5-1 progressing :: 1080
Finished! Spatter :: 54, Non-spatter :: 130
5-2 progressing :: 1081
Finished! Spatter :: 31, Non-spatter :: 108
6-1 progressing :: 2163
Finished! Spatter :: 147, Non-spatter :: 227
6-2 progressing :: 2162
Finished! Spatter :: 157, Non-spatter :: 195
7-1 progressing :: 540
Finished! Spatter :: 71, Non-spatter :: 38
7-2 progressing :: 540
Finished! Spatter :: 65, Non-spatter :: 41
8-1 progressing :: 1082
Finished! Spatter :: 42, Non-spatt

#check dataset

In [ ]:
import pandas as pd
file_train = pd.read_csv(filename)
train = file_train.values
print(train.shape)
file_test = pd.read_csv(filename_test)
test = file_test.values
print(test.shape)

(3952, 2)
(710, 2)


#20220603

In [22]:
dataset = Dataset(x_patch = 3, y_patch = 3, height = 80,width = 80,
                date = "20220603",
                src_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20220603',
                dir_concatImg = "concat_vit"
                )

#make dataset one by one
for i in range(1,13):
  for j in range(1,3):
    dataset.make(filename=filename,filename_test=filename_test,div = i,sec = j)

1-1 progressing :: 540
Finished! Spatter :: 0, Non-spatter :: 67
1-2 progressing :: 533
Finished! Spatter :: 0, Non-spatter :: 32
2-1 progressing :: 1079
Finished! Spatter :: 9, Non-spatter :: 123
2-2 progressing :: 1080
Finished! Spatter :: 0, Non-spatter :: 109
3-1 progressing :: 2162
Finished! Spatter :: 9, Non-spatter :: 233
3-2 progressing :: 2163
Finished! Spatter :: 18, Non-spatter :: 249
4-1 progressing :: 457
Finished! Spatter :: 19, Non-spatter :: 46
4-2 progressing :: 540
Finished! Spatter :: 18, Non-spatter :: 60
5-1 progressing :: 1079
Finished! Spatter :: 0, Non-spatter :: 133
5-2 progressing :: 1080
Finished! Spatter :: 18, Non-spatter :: 126
6-1 progressing :: 2162
Finished! Spatter :: 18, Non-spatter :: 274
6-2 progressing :: 2162
Finished! Spatter :: 35, Non-spatter :: 253
7-1 progressing :: 541
Finished! Spatter :: 21, Non-spatter :: 61
7-2 progressing :: 540
Finished! Spatter :: 28, Non-spatter :: 70
8-1 progressing :: 1080
Finished! Spatter :: 0, Non-spatter :: 125

In [23]:
import pandas as pd
file_train = pd.read_csv(filename)
train = file_train.values
print(train.shape)
file_test = pd.read_csv(filename_test)
test = file_test.values
print(test.shape)

(8085, 2)
(858, 2)


#20221020

In [24]:
dataset = Dataset(x_patch = 3, y_patch = 3, height = 80,width = 80,
                date = "20221020",
                src_dir = r'C:\Users\Furukawa\Documents\TechAssistant\dataset\imgs\20221020',
                dir_concatImg = "concat_vit"
                )

#make dataset one by one
for i in range(1,13):
  for j in range(1,3):
    dataset.make(filename=filename,filename_test=filename_test,div = i,sec = j)

1-1 progressing :: 244


C:\Users\Furukawa\AppData\Local\Temp\ipykernel_63684\4225067730.py:86: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20221020\video\XIMEA_221020_21_trim_spatter.txt"
  self.spatter_data_1_1 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,21)), dtype='int').tolist()
C:\Users\Furukawa\AppData\Local\Temp\ipykernel_63684\4225067730.py:87: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20221020\video\XIMEA_221020_22_trim_spatter.txt"
  self.spatter_data_1_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,22)), dtype='int').tolist()
C:\Users\Furukawa\AppData\Local\Temp\ipykernel_63684\4225067730.py:89: UserWarning: loadtxt: input contained no data: "C:\Users\Furukawa\Documents\TechAssistant\Video\20221020\video\XIMEA_221020_24_trim_spatter.txt"
  self.spatter_data_2_2 = np.loadtxt(os.path.join(dir,'{}_{}_trim_spatter.txt'.format(head,24)), dtype='int').toli

Finished! Spatter :: 0, Non-spatter :: 26
1-2 progressing :: 448
Finished! Spatter :: 0, Non-spatter :: 58
2-1 progressing :: 895
Finished! Spatter :: 0, Non-spatter :: 101
2-2 progressing :: 245
Finished! Spatter :: 0, Non-spatter :: 34
3-1 progressing :: 448
Finished! Spatter :: 0, Non-spatter :: 54
3-2 progressing :: 863
Finished! Spatter :: 0, Non-spatter :: 93
4-1 progressing :: 235
Finished! Spatter :: 0, Non-spatter :: 31
4-2 progressing :: 430
Finished! Spatter :: 8, Non-spatter :: 46
5-1 progressing :: 857
Finished! Spatter :: 9, Non-spatter :: 122
5-2 progressing :: 233
Finished! Spatter :: 7, Non-spatter :: 34
6-1 progressing :: 312
Finished! Spatter :: 0, Non-spatter :: 46
6-2 progressing :: 880
Finished! Spatter :: 0, Non-spatter :: 122
7-1 progressing :: 236
Finished! Spatter :: 6, Non-spatter :: 22
7-2 progressing :: 434
Finished! Spatter :: 18, Non-spatter :: 34
8-1 progressing :: 866
Finished! Spatter :: 0, Non-spatter :: 79
8-2 progressing :: 240
Finished! Spatter :: 

In [25]:
import pandas as pd
file_train = pd.read_csv(filename)
train = file_train.values
print(train.shape)
file_test = pd.read_csv(filename_test)
test = file_test.values
print(test.shape)

(9563, 2)
(1003, 2)


# total dataset

In [26]:
import pandas as pd
file_train = pd.read_csv(filename)
train = file_train.values
labels_train = train[:,0]
print(train.shape)
file_test = pd.read_csv(filename_test)
test = file_test.values
labels_test = test[:,0]
print(test.shape)
def count_spatter(labels):
    count = 0
    for i in range(labels.shape[0]):
        if labels[i] == 1:
            count += 1
    print(f"Spatter/toal : {count}/{labels.shape[0]}")

print("train data")
count_spatter(labels_train)
print("test data")
count_spatter(labels_test)

(9563, 2)
(1003, 2)
train data
Spatter/toal : 2298/9563
test data
Spatter/toal : 241/1003
